In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

import tensorflow as tf

## Carga del dataset

In [ ]:
# Asegúrate de que este notebook esté en la misma carpeta que Ecommerce.csv
df = pd.read_csv('Ecommerce.csv')
df.head()

## Formulación del modelo

Usaremos como variables explicativas:

- `Avg. Session Length`
- `Time on App`
- `Time on Website`
- `Length of Membership`

La variable objetivo será:

- `Yearly Amount Spent`

In [ ]:
feature_cols = ['Avg. Session Length', 'Time on App', 'Time on Website', 'Length of Membership']
target_col = 'Yearly Amount Spent'

X = df[feature_cols].values
y = df[target_col].values

## División train/test y escalamiento de features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled.shape, X_test_scaled.shape

## Definición del perceptrón regresivo con TensorFlow / Keras

Un perceptrón regresivo es un modelo lineal con una sola neurona y activación lineal.

En Keras lo representamos como una red `Sequential` con una sola capa `Dense` de 1 neurona y `activation='linear'`.

In [ ]:
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=(X_train_scaled.shape[1],), activation='linear')
])

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
    loss='mse',
    metrics=['mse']
)

model.summary()

## Entrenamiento del modelo

In [ ]:
history = model.fit(
    X_train_scaled, y_train,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    verbose=0
)

len(history.history['loss'])

## Haciendo predicciones

In [ ]:
y_train_pred = model.predict(X_train_scaled).flatten()
y_test_pred = model.predict(X_test_scaled).flatten()

y_train_pred[:5], y_test_pred[:5]

In [ ]:
# Comparación de algunos valores reales vs predichos (test)
comparison = pd.DataFrame({
    'Real': y_test[:10],
    'Predicho': y_test_pred[:10]
})
comparison

## Evaluando el desempeño del modelo

In [ ]:
# Métricas en entrenamiento
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_train_pred)

# Métricas en test
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_test_pred)

print('Entrenamiento:')
print(f'MSE:  {mse_train:.2f}')
print(f'RMSE: {rmse_train:.2f}')
print(f'R2:   {r2_train:.4f}')

print('\nTest:')
print(f'MSE:  {mse_test:.2f}')
print(f'RMSE: {rmse_test:.2f}')
print(f'R2:   {r2_test:.4f}')

In [ ]:
# Gráfico de valores reales vs predichos (test)
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_test_pred, alpha=0.7)
plt.xlabel('Yearly Amount Spent real')
plt.ylabel('Yearly Amount Spent predicho')
plt.title('Perceptrón regresivo (TensorFlow) - Test')

min_val = min(y_test.min(), y_test_pred.min())
max_val = max(y_test.max(), y_test_pred.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--')
plt.show()

## Error (loss) por epochs

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.figure(figsize=(8,4))
plt.plot(epochs, loss, label='Entrenamiento')
plt.plot(epochs, val_loss, label='Validación')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.title('Evolución del error (MSE) durante el entrenamiento')
plt.legend()
plt.show()

### Comentarios

- Este modelo es equivalente al perceptrón regresivo implementado "a mano", pero usando TensorFlow para el cálculo automático de gradientes y la actualización de pesos.
- Puedes modificar parámetros como `learning_rate`, `epochs` y `batch_size` para ver cómo cambia el desempeño.
- También podrías comparar con otros optimizadores (`Adam`, `RMSprop`, etc.).